In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import csv
import os

NUM_OF_YEARS = 10
CSV_FILENAME = "mse_data.csv"

# Функција за преземање на тикерите
def fetch_tikeri_bs():
    url = 'https://www.mse.mk/mk/stats/symbolhistory/kmb'
    response = requests.get(url)
    
    if response.status_code != 200:
        print("Не успеав да ја превземам страната.")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    dropdown = soup.find('select', {'id': 'Code'})
    options = dropdown.find_all('option') if dropdown else []
    
    # Филтрирај ги тикерите кои немаат броеви
    return [option['value'] for option in options if option['value'].isalpha()]

# Функција за преземање на податоци за тикер
def fetch_data_for_ticker(ticker, date_from, date_to):
    base_url = f"https://www.mse.mk/mk/stats/symbolhistory/{ticker.lower()}"
    params = {
        "FromDate": date_from.strftime("%d.%m.%Y"),
        "ToDate": date_to.strftime("%d.%m.%Y"),
    }
    
    response = requests.get(base_url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.select_one('#resultsTable > tbody')
    data = []
    
    if table:
        rows = table.find_all('tr')
        for row in rows:
            data_row = [cell.text.strip() for cell in row.find_all('td')]
            if data_row:
                data_row.insert(0, ticker)  # Додај го тикерот како прва колона
                data.append(data_row)
    return data

# Функција за зачувување на податоци во CSV фајл
def save_to_csv(data):
    file_exists = os.path.isfile(CSV_FILENAME)
    
    with open(CSV_FILENAME, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Додај ја заглавната линија ако фајлот е нов
        if not file_exists:
            writer.writerow(["Tiker", "Datum", "Otvorena cena", "Najgolema cena", "Najmala cena", "Zatvorena cena", "Volumen", "Promet"])
        
        writer.writerows(data)

# Главна функција
def main():
    # Преземи ги сите тикери
    tickers = fetch_tikeri_bs()
    print(f"Пронајдени тикери: {tickers}")

    date_to = datetime.now()
    date_from = date_to - timedelta(days=365 * NUM_OF_YEARS)

    all_data = []

    for ticker in tickers:
        print(f"Преземам податоци за {ticker}...")
        data = fetch_data_for_ticker(ticker, date_from, date_to)
        
        if data:
            all_data.extend(data)
        else:
            print(f"Нема достапни податоци за {ticker}.")
    
    # Зачувај ги податоците во CSV ако се достапни
    if all_data:
        save_to_csv(all_data)
        print("Сите податоци се зачувани во mse_data.csv.")
    else:
        print("Нема нови податоци за зачувување.")

# Повикај ја главната функција
main()


Пронајдени тикери: ['ADIN', 'ALK', 'ALKB', 'AMEH', 'APTK', 'ATPP', 'AUMK', 'BANA', 'BGOR', 'BIKF', 'BIM', 'BLTU', 'CBNG', 'CDHV', 'CEVI', 'CKB', 'CKBKO', 'DEBA', 'DIMI', 'EDST', 'ELMA', 'ELNC', 'ENER', 'ENSA', 'EUHA', 'EUMK', 'EVRO', 'FAKM', 'FERS', 'FKTL', 'FROT', 'FUBT', 'GALE', 'GDKM', 'GECK', 'GECT', 'GIMS', 'GRDN', 'GRNT', 'GRSN', 'GRZD', 'GTC', 'GTRG', 'IJUG', 'INB', 'INHO', 'INOV', 'INPR', 'INTP', 'JAKO', 'JUSK', 'KARO', 'KDFO', 'KJUBI', 'KKST', 'KLST', 'KMB', 'KMPR', 'KOMU', 'KONF', 'KONZ', 'KORZ', 'KPSS', 'KULT', 'KVAS', 'LAJO', 'LHND', 'LOTO', 'LOZP', 'MAGP', 'MAKP', 'MAKS', 'MB', 'MERM', 'MKSD', 'MLKR', 'MODA', 'MPOL', 'MPT', 'MPTE', 'MTUR', 'MZHE', 'MZPU', 'NEME', 'NOSK', 'OBPP', 'OILK', 'OKTA', 'OMOS', 'OPFO', 'OPTK', 'ORAN', 'OSPO', 'OTEK', 'PELK', 'PGGV', 'PKB', 'POPK', 'PPIV', 'PROD', 'PROT', 'PTRS', 'RADE', 'REPL', 'RIMI', 'RINS', 'RZEK', 'RZIT', 'RZIZ', 'RZLE', 'RZLV', 'RZTK', 'RZUG', 'RZUS', 'SBT', 'SDOM', 'SIL', 'SKON', 'SKP', 'SLAV', 'SNBT', 'SNBTO', 'SOLN', 'SPAZ'